In [2]:
!pip install requests beautifulsoup4


In [3]:
import requests
from bs4 import BeautifulSoup

def get_weather_from_google(city: str):
    # Create URL
    url = "https://www.google.com/search?q=" + "weather" + city

    # Add headers (important on Colab to avoid getting blocked)
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/120.0.0.0 Safari/537.36"
        )
    }

    # Get raw HTML
    response = requests.get(url, headers=headers)
    html = response.content

    # Parse HTML
    soup = BeautifulSoup(html, 'html.parser')

    # Extract temperature
    temp = soup.find('div', attrs={'class': 'BNeawe iBp4i AP7Wnd'})
    if temp:
        temp = temp.text
    else:
        temp = "N/A"

    # Extract time + sky description
    str_ = soup.find('div', attrs={'class': 'BNeawe tAd8D AP7Wnd'})
    if str_:
        data = str_.text.split('\n')
        time = data[0] if len(data) > 0 else "N/A"
        sky = data[1] if len(data) > 1 else "N/A"
    else:
        time = "N/A"
        sky = "N/A"

    # Extract other details (wind, humidity, etc.)
    listdiv = soup.findAll('div', attrs={'class': 'BNeawe s3v9rd AP7Wnd'})
    other_data = "N/A"
    if len(listdiv) >= 6:  # index 5 exists
        strd = listdiv[5].text
        pos = strd.find('Wind')
        if pos != -1:
            other_data = strd[pos:]

    # Print results like in the article
    print(f"City: {city}")
    print("Temperature is:", temp)
    print("Time:", time)
    print("Sky Description:", sky)
    print(other_data)


# ---- Test it once ----
get_weather_from_google("lucknow")


City: lucknow
Temperature is: N/A
Time: N/A
Sky Description: N/A
N/A


/tmp/ipython-input-766147836.py:42: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  listdiv = soup.findAll('div', attrs={'class': 'BNeawe s3v9rd AP7Wnd'})


In [4]:
city_input = input("Enter city name: ")
get_weather_from_google(city_input)


Enter city name: Delhi
City: Delhi
Temperature is: N/A
Time: N/A
Sky Description: N/A
N/A


/tmp/ipython-input-766147836.py:42: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  listdiv = soup.findAll('div', attrs={'class': 'BNeawe s3v9rd AP7Wnd'})


In [5]:
import requests

# Get IP-based location info
res = requests.get('https://ipinfo.io/')
data = res.json()

# Extract city
citydata = data.get('city', 'Unknown')
print("Current Location (from IP):", citydata)

# Get schematic weather from wttr.in
url = f'https://wttr.in/{citydata}'
res_weather = requests.get(url)

print(res_weather.text)


Current Location (from IP): North Charleston
Weather report: North Charleston

     \  /       Partly cloudy
   _ /"".-.     64 °F          
     \_(   ).   → 6 mph        
     /(___(__)  9 mi           
                0.0 in         
                                                       ┌─────────────┐                                                       
┌──────────────────────────────┬───────────────────────┤  Wed 26 Nov ├───────────────────────┬──────────────────────────────┐
│            Morning           │             Noon      └──────┬──────┘     Evening           │             Night            │
├──────────────────────────────┼──────────────────────────────┼──────────────────────────────┼──────────────────────────────┤
│     \   /     Sunny          │     \   /     Sunny          │     \   /     Clear          │    \  /       Partly cloudy  │
│      .-.      66 °F          │      .-.      +75(78) °F     │      .-.      +75(78) °F     │  _ /"".-.     64 °F          │
│   ― (

In [6]:
import requests

city = input("Enter city for wttr.in weather: ")

url = f'https://wttr.in/{city}'
res = requests.get(url)

print(res.text)


Enter city for wttr.in weather: Uganda
Weather report: Uganda

      \   /     Clear
       .-.      66 °F          
    ― (   ) ―   ↑ 2 mph        
       `-’      6 mi           
      /   \     0.0 in         
                                                       ┌─────────────┐                                                       
┌──────────────────────────────┬───────────────────────┤  Thu 27 Nov ├───────────────────────┬──────────────────────────────┐
│            Morning           │             Noon      └──────┬──────┘     Evening           │             Night            │
├──────────────────────────────┼──────────────────────────────┼──────────────────────────────┼──────────────────────────────┤
│     \   /     Sunny          │    \  /       Partly Cloudy  │     \   /     Sunny          │     \   /     Clear          │
│      .-.      78 °F          │  _ /"".-.     91 °F          │      .-.      78 °F          │      .-.      69 °F          │
│   ― (   ) ―   ↑ 3 mph        